In [62]:

import re
textpath = "data/nlp.txt"


## 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [7]:
with open(textpath) as text, open("out/50.txt", "w") as f:
    f.write(re.sub(r"(?<=[.;:?!])\s(?=[A-Z])", "\n", text.read()))

## 51. 単語の切り出し
空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [14]:
with open("out/50.txt") as text, open("out/51.txt", "w") as f:
    space2nl = re.sub("\s", "\n", text.read())
    new_text = re.sub("\.\n", ".\n\n", space2nl)
    f.write(new_text)

## 52. ステミング
51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

In [19]:
from nltk.stem.porter import PorterStemmer as PS
ps = PS()

with open("out/51.txt") as words, open("out/52.txt", "w") as f:
    for word in words:
        f.write(word.strip() + "\t" + ps.stem(word.strip()) + "\n")

## 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

### memo
`java -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -file nlp.txt`

- root
    0. docID
    1. sentences
        - sentence
            - tokens
                - token 
                    0. word
                    1. lemma
                    2. CharacterOffsetBegin
                    3. CharacterOffsetEnd
                    4. POS
                    5. NER
                    6. Speaker    
            - dependencies
                - dep
                    0. governor
                    1. dependent
    2. coreferences
        - coreference
            - mention
                0. sentence
                1. start
                2. end
                3. head
                4. text
        

In [32]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/nlp.txt.xml")
root = tree.getroot()

for token in root.iter('token'):
    1
    #print(token[0].text)

for a in root.itertext(): 
    print(a)


  

    
nlp.txt

    

      

        

          

            
Natural

            
natural

            
0

            
7

            
JJ

            
O

            
PER0

          

          

            
language

            
language

            
8

            
16

            
NN

            
O

            
PER0

          

          

            
processing

            
processing

            
17

            
27

            
NN

            
O

            
PER0

          

          

            
From

            
from

            
28

            
32

            
IN

            
O

            
PER0

          

          

            
Wikipedia

            
Wikipedia

            
33

            
42

            
NNP

            
ORGANIZATION

            
PER0

          

          

            
,

            
,

            
42

            
43

            
,

            
O

            
PER0

          

          

            
the

 

395

            
403

            
VBG

            
O

            
PER0

          

          

            
computers

            
computer

            
404

            
413

            
NNS

            
O

            
PER0

          

          

            
to

            
to

            
414

            
416

            
TO

            
O

            
PER0

          

          

            
derive

            
derive

            
417

            
423

            
VB

            
O

            
PER0

          

          

            
meaning

            
meaning

            
424

            
431

            
NN

            
O

            
PER0

          

          

            
from

            
from

            
432

            
436

            
IN

            
O

            
PER0

          

          

            
human

            
human

            
437

            
442

            
JJ

            
O

            
PER0

      


          

            
which

            
which

            
715

            
720

            
WDT

            
O

            
PER0

          

          

            
proposed

            
propose

            
721

            
729

            
VBD

            
O

            
PER0

          

          

            
what

            
what

            
730

            
734

            
WP

            
O

            
PER0

          

          

            
is

            
be

            
735

            
737

            
VBZ

            
O

            
PER0

          

          

            
now

            
now

            
738

            
741

            
RB

            
DATE

            
PRESENT_REF

            
PER0

            
PRESENT_REF

          

          

            
called

            
call

            
742

            
748

            
VBN

            
O

            
PER0

          

          

            
the

     

          

          

            
problem

            
,

          

          

            
translation

            
machine

          

          

            
problem

            
translation

          

          

            
problem

            
would

          

          

            
problem

            
be

          

          

            
problem

            
a

          

          

            
problem

            
solved

          

          

            
claimed

            
problem

          

          

            
claimed

            
.

          

        

        

          

            
ROOT

            
claimed

          

          

            
authors

            
The

          

          

            
claimed

            
authors

          

          

            
problem

            
that

          

          

            
years

            
within

          

          

            
years

            
th

          

          

            
research

            
Little

          

          

            
research

            
further

          

          

            
conducted

            
research

          

          

            
translation

            
in

          

          

            
translation

            
machine

          

          

            
research

            
translation

          

          

            
conducted

            
was

          

          

            
1980s

            
until

          

          

            
1980s

            
the

          

          

            
1980s

            
late

          

          

            
conducted

            
1980s

          

          

            
1980s

            
,

          

          

            
developed

            
when

          

          

            
systems

            
the

          

          

            
systems

            
first



...

            
DT

            
O

            
PER0

          

          

            
where

            
where

            
6961

            
6966

            
WRB

            
O

            
PER0

          

          

            
the

            
the

            
6967

            
6970

            
DT

            
O

            
PER0

          

          

            
effort

            
effort

            
6971

            
6977

            
NN

            
O

            
PER0

          

          

            
should

            
should

            
6978

            
6984

            
MD

            
O

            
PER0

          

          

            
be

            
be

            
6985

            
6987

            
VB

            
O

            
PER0

          

          

            
directed

            
direct

            
6988

            
6996

            
VBN

            
O

            
PER0

          

          




            
input

          

          

            
containing

            
-LRB-

          

          

            
containing

            
e.g.

          

          

            
input

            
containing

          

          

            
containing

            
words

          

          

            
words

            
or

          

          

            
containing

            
structures

          

          

            
words

            
structures

          

          

            
seen

            
that

          

          

            
seen

            
have

          

          

            
seen

            
not

          

          

            
seen

            
been

          

          

            
words

            
seen

          

          

            
seen

            
before

          

          

            
containing

            
-RRB-

          

          

            
robust

            

          

            
ROOT

            
made

          

          

            
made

            
Systems

          

          

            
Systems

            
based

          

          

            
learning

            
on

          

          

            
learning

            
automatically

          

          

            
based

            
learning

          

          

            
rules

            
the

          

          

            
learning

            
rules

          

          

            
made

            
can

          

          

            
made

            
be

          

          

            
accurate

            
more

          

          

            
made

            
accurate

          

          

            
supplying

            
simply

          

          

            
supplying

            
by

          

          

            
accurate

            
supplying

          

          

  

7949

            
TO

            
O

            
PER0

          

          

            
machine-learning

            
machine-learning

            
7950

            
7966

            
JJ

            
O

            
PER0

          

          

            
systems

            
system

            
7967

            
7974

            
NNS

            
O

            
PER0

          

          

            
simply

            
simply

            
7975

            
7981

            
RB

            
O

            
PER0

          

          

            
requires

            
require

            
7982

            
7990

            
VBZ

            
O

            
PER0

          

          

            
a

            
a

            
7991

            
7992

            
DT

            
O

            
PER0

          

          

            
corresponding

            
corresponding

            
7993

            
8006

            
JJ

           

            
CoNLL

            
conference

          

          

            
known

            
CoNLL

          

          

            
Learning

            
CoNLL

          

          

            
CoNLL

            
and

          

          

            
SIGNLL

            
peak

          

          

            
SIGNLL

            
body

          

          

            
Learning

            
SIGNLL

          

          

            
CoNLL

            
SIGNLL

          

          

            
sponsored

            
are

          

          

            
ACL

            
by

          

          

            
sponsored

            
ACL

          

          

            
sponsored

            
,

          

          

            
sponsored

            
recognizing

          

          

            
recognizing

            
also

          

          

            
links

            
their

          

          

            
re

## 54. 品詞タグ付け
Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [14]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/nlp.txt.xml")
root = tree.getroot()

for token in root.iter('token'):
    line = '\t'.join(map(lambda i: token[i].text, [0,1,4]))
    #print(line)

## 55. 固有表現抽出
入力文中の人名をすべて抜き出せ．

In [15]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/nlp.txt.xml")
root = tree.getroot()

for token in root.iter('token'):
    if token[5].text == "PERSON":
        print(token[0].text)

Alan
Turing
Joseph
Weizenbaum
MARGIE
Schank
Wilensky
Meehan
Lehnert
Carbonell
Lehnert
Racter
Jabberwacky
Moore


## 56. 共参照解析
Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

### memo
- mention
                0. sentence 何文目
                1. start (token)
                2. end (token)
                3. head
                4. text

sentence, token ID のcharacteroffsetbegin / end の範囲を置換する

In [105]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/nlp.txt.xml")
root = tree.getroot()

sub_list = []
for coreference in root[0][2]:
    for mention in coreference:
        if mention.attrib: # representative
            rep =  mention[4].text
        else:
            _sid, _ts, _te, _, ment = [attr.text for attr in mention]
            sentence_id, token_start, token_end = int(_sid)-1, int(_ts)-1, int(_te)-2
            start = int(root[0][1][sentence_id][0][token_start][2].text)
            end = int(root[0][1][sentence_id][0][token_end][3].text)
            if rep != ment:
                sub_list.append((start, end, rep, ment)) 

sub_list.sort(key= lambda t: -t[1])

with open(textpath) as text, open("out/56.txt" , 'w') as out:
    string = text.read()
    for start, end, rep, ment in sub_list:
        repl = rep + '(' + ment + ')'
        string = string[:start] + re.sub(ment, repl, string[start:],1)
        
    out.write(string)

## 57. 係り受け解析
Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

## 58. タプルの抽出
Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）に基づき，「主語 述語 目的語」の組をタブ区切り形式で出力せよ．ただし，主語，述語，目的語の定義は以下を参考にせよ．

- 述語: nsubj関係とdobj関係の子（dependant）を持つ単語
- 主語: 述語からnsubj関係にある子（dependent）
- 目的語: 述語からdobj関係にある子（dependent）

## 59. S式の解析
Stanford Core NLPの句構造解析の結果（S式）を読み込み，文中のすべての名詞句（NP）を表示せよ．入れ子になっている名詞句もすべて表示すること．